In [ ]:
import torch

print("CUDA available:", torch.cuda.is_available())
print("GPU:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU detected")


## preproccssing 

In [ ]:
import os
import cv2
import random
import shutil
import numpy as np
from sklearn.model_selection import train_test_split

# Set random seed for reproducibility
random.seed(42)

# Define paths
input_dir = '/path/to/UCF101'  # Update with your UCF101 dataset path
output_dir = '/path/to/UCF101_processed'  # Update with your output path
frame_output_dir = os.path.join(output_dir, 'frames')  # Frames output directory

# Create output directories
os.makedirs(output_dir, exist_ok=True)
os.makedirs(frame_output_dir, exist_ok=True)

def resize_videos(input_dir, output_dir, width=224, height=224):
    for class_folder in os.listdir(input_dir):
        class_path = os.path.join(input_dir, class_folder)
        if os.path.isdir(class_path):
            output_class_path = os.path.join(output_dir, class_folder)
            os.makedirs(output_class_path, exist_ok=True)
            
            for video_file in os.listdir(class_path):
                video_path = os.path.join(class_path, video_file)
                if video_file.endswith(('.mp4', '.avi')):
                    cap = cv2.VideoCapture(video_path)
                    fourcc = cv2.VideoWriter_fourcc(*'XVID')
                    out = cv2.VideoWriter(os.path.join(output_class_path, video_file), fourcc, 30, (width, height))
                    
                    while cap.isOpened():
                        ret, frame = cap.read()
                        if not ret:
                            break
                        resized_frame = cv2.resize(frame, (width, height))
                        out.write(resized_frame)
                        
                    cap.release()
                    out.release()
    print("Videos resized and saved to:", output_dir)

def extract_frames(video_path, output_folder, frame_rate=1):
    os.makedirs(output_folder, exist_ok=True)
    cap = cv2.VideoCapture(video_path)
    frame_count = 0
    
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        if frame_count % frame_rate == 0:  # Save every nth frame
            cv2.imwrite(os.path.join(output_folder, f"{os.path.basename(video_path).split('.')[0]}_frame_{frame_count}.jpg"), frame)
        frame_count += 1
        
    cap.release()
    print(f"Frames extracted and saved to: {output_folder}")

def augment_frame(frame):
    # Example augmentation: Flip horizontally
    if random.random() > 0.5:
        frame = cv2.flip(frame, 1)  # Flip horizontally
    return frame

def prepare_frames_for_augmentation(input_frames_dir, output_frames_dir):
    os.makedirs(output_frames_dir, exist_ok=True)

    for frame_file in os.listdir(input_frames_dir):
        frame_path = os.path.join(input_frames_dir, frame_file)
        frame = cv2.imread(frame_path)
        augmented_frame = augment_frame(frame)
        cv2.imwrite(os.path.join(output_frames_dir, frame_file), augmented_frame)

def split_dataset(input_dir, test_size=0.2):
    all_classes = os.listdir(input_dir)
    train_data, val_data, test_data = [], [], []
    
    for class_folder in all_classes:
        class_path = os.path.join(input_dir, class_folder)
        if os.path.isdir(class_path):
            videos = [os.path.join(class_folder, video) for video in os.listdir(class_path)]
            train_videos, temp_videos = train_test_split(videos, test_size=test_size, random_state=42)
            val_videos, test_videos = train_test_split(temp_videos, test_size=0.5, random_state=42)
            
            train_data.extend(train_videos)
            val_data.extend(val_videos)
            test_data.extend(test_videos)
    
    return train_data, val_data, test_data

# Step 1: Resize Videos
resize_videos(input_dir, output_dir)

# Step 2: Extract Frames from Resized Videos
for class_folder in os.listdir(output_dir):
    class_path = os.path.join(output_dir, class_folder)
    if os.path.isdir(class_path):
        for video_file in os.listdir(class_path):
            video_path = os.path.join(class_path, video_file)
            extract_frames(video_path, frame_output_dir)

# Step 3: Prepare Frames for Augmentation (optional)
augmented_frame_output_dir = os.path.join(output_dir, 'augmented_frames')
prepare_frames_for_augmentation(frame_output_dir, augmented_frame_output_dir)

# Step 4: Split Dataset
train_data, val_data, test_data = split_dataset(output_dir)

# Display the results
print("Train Data:", train_data)
print("Validation Data:", val_data)
print("Test Data:", test_data)
